In [14]:
!pip install pandas
!pip install numpy
!pip install matplotlib as
!pip install scikit-learn
!pip install statsmodels pmdarima
!pip install tbats
!pip install prophet
!pip install statsmodels

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from tbats import TBATS
from statsmodels.tsa.forecasting.theta import ThetaModel

# -----------------------------
# 1. 데이터 불러오기 및 파싱
# -----------------------------


# CSV 파일 읽기 (헤더 없음)
df = pd.read_csv(file_path, header=None)

# 15번째 줄부터 끝까지, 첫 번째 열만 추출
data_lines = df.iloc[15:, 0].reset_index(drop=True)

# 시계열 파싱 함수는 그대로 사용 가능
def parse_series(line):
    parts = line.split(':')
    series_id = parts[0]
    start_date = parts[1].split(' ')[0]
    values = list(map(float, parts[2].split(',')))

    # 분기 인덱스 생성
    index = pd.date_range(start=start_date, periods=len(values), freq='Q')
    series = pd.Series(values, index=index, name=series_id)
    return series

# 전체 시계열 데이터 리스트 생성
all_series = [parse_series(line) for line in data_lines]
print(f"총 시계열 수: {len(all_series)}")

# -----------------------------
# 2. 모델 학습 및 평가 함수 정의
# -----------------------------

# 성능 평가 함수 (MAE, RMSE)
def evaluate_forecast(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

# 개별 시계열에 대한 모델 학습/예측/평가
def forecast_models(series):
    results = {}
    train = series[:-8]
    test = series[-8:]

    # SES
    try:
        ses_model = SimpleExpSmoothing(train).fit()
        ses_forecast = ses_model.forecast(len(test))
        results['SES'] = evaluate_forecast(test, ses_forecast)
    except Exception as e:
        results['SES'] = (np.nan, np.nan)

    # Theta
    try:
        theta_model = ThetaModel(train)
        theta_forecast = theta_model.forecast(len(test))
        results['Theta'] = evaluate_forecast(test, theta_forecast)
    except Exception as e:
        results['Theta'] = (np.nan, np.nan)

    # TBATS
    try:
        tbats_estimator = TBATS(seasonal_periods=[4])
        tbats_model = tbats_estimator.fit(train)
        tbats_forecast = tbats_model.forecast(steps=len(test))
        results['TBATS'] = evaluate_forecast(test, tbats_forecast)
    except Exception as e:
        results['TBATS'] = (np.nan, np.nan)

    # ETS
    try:
        ets_model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=4).fit()
        ets_forecast = ets_model.forecast(len(test))
        results['ETS'] = evaluate_forecast(test, ets_forecast)
    except Exception as e:
        results['ETS'] = (np.nan, np.nan)

    # ARIMA (SARIMA)
    try:
        sarima_model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,4)).fit(disp=False)
        sarima_forecast = sarima_model.forecast(steps=len(test))
        results['SARIMA'] = evaluate_forecast(test, sarima_forecast)
    except Exception as e:
        results['SARIMA'] = (np.nan, np.nan)

    # Prophet
    try:
        prophet_df = train.reset_index()
        prophet_df.columns = ['ds', 'y']
        prophet_model = Prophet(seasonality_mode='additive', yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
        prophet_model.add_seasonality(name='quarterly', period=4, fourier_order=3)
        prophet_model.fit(prophet_df)
        future = prophet_model.make_future_dataframe(periods=len(test), freq='Q')
        forecast = prophet_model.predict(future)
        prophet_forecast = forecast.iloc[-len(test):]['yhat'].values
        results['Prophet'] = evaluate_forecast(test, prophet_forecast)
    except Exception as e:
        results['Prophet'] = (np.nan, np.nan)

    return results

# -----------------------------
# 3. 전체 시계열에 대한 루프 실행
# -----------------------------

all_results = []

for idx, series in enumerate(all_series):
    print(f"[{idx+1}/{len(all_series)}] 시계열 ID: {series.name} 분석 중...")
    res = forecast_models(series)
    all_results.append(res)

# -----------------------------
# 4. 결과 정리 및 평균 성능 출력
# -----------------------------

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(all_results)

# 모델별 MAE 평균
avg_mae = result_df.applymap(lambda x: x[0] if isinstance(x, tuple) else np.nan).mean()
# 모델별 RMSE 평균
avg_rmse = result_df.applymap(lambda x: x[1] if isinstance(x, tuple) else np.nan).mean()

print("모델별 평균 MAE:")
print(avg_mae)

print("\n모델별 평균 RMSE:")
print(avg_rmse)

# -----------------------------
# 5. 결과 시각화 (선택)
# -----------------------------

plt.figure(figsize=(10,5))
plt.bar(avg_mae.index, avg_mae.values)
plt.title('평균 MAE 비교')
plt.ylabel('MAE')
plt.show()

plt.figure(figsize=(10,5))
plt.bar(avg_rmse.index, avg_rmse.values)
plt.title('평균 RMSE 비교')
plt.ylabel('RMSE')
plt.show()


총 시계열 수: 427
[1/427] 시계열 ID: T1 분석 중...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/_6keke8g.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/b7rmq56z.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=29362', 'data', 'file=/tmp/tmp_tkrtbp6/_6keke8g.json', 'init=/tmp/tmp_tkrtbp6/b7rmq56z.json', 'output', 'file=/tmp/tmp_tkrtbp6/prophet_modelfl3e5r1y/prophet_model-20250313074607.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:46:07 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:46:07 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


[2/427] 시계열 ID: T2 분석 중...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/0wkm62d_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/k6ccb_8z.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=74111', 'data', 'file=/tmp/tmp_tkrtbp6/0wkm62d_.json', 'init=/tmp/tmp_tkrtbp6/k6ccb_8z.json', 'output', 'file=/tmp/tmp_tkrtbp6/prophet_modelv0bfxwk0/prophet_model-20250313074806.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:48:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:48:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


[3/427] 시계열 ID: T3 분석 중...


...

[31/427] 시계열 ID: T31 분석 중...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/gyiafk76.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/4ldob97d.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=76604', 'data', 'file=/tmp/tmp_tkrtbp6/gyiafk76.json', 'init=/tmp/tmp_tkrtbp6/4ldob97d.json', 'output', 'file=/tmp/tmp_tkrtbp6/prophet_modelhqvvajmr/prophet_model-20250313083148.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:31:48 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:31:49 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


[32/427] 시계열 ID: T32 분석 중...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/f0udom2g.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp_tkrtbp6/zjs4xxqq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67966', 'data', 'file=/tmp/tmp_tkrtbp6/f0udom2g.json', 'init=/tmp/tmp_tkrtbp6/zjs4xxqq.json', 'output', 'file=/tmp/tmp_tkrtbp6/prophet_modeludyml5cv/prophet_model-20250313083344.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:33:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:33:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


[33/427] 시계열 ID: T33 분석 중...
